In [1]:
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split



División de imagenes en malignas y benignas

In [ ]:
df = pd.read_csv("../data/raw/HAM10000_metadata.csv")
# Definir clases benignas
clases_benignas = {'nv', 'bkl', 'df'}

# Añadir columna binaria
df['etiqueta'] = df['dx'].apply(lambda x: 'benignas' if x in clases_benignas else 'malignas')

# Dividir en train y test (estratificado por etiqueta)
df_train, df_test = train_test_split(df, test_size=0.1, stratify=df['etiqueta'], random_state=42)

# Rutas
carpeta_origen = '../data/raw/imagenes'
carpeta_destino = '../data/processed'

for conjunto, df_conjunto in [('train', df_train), ('test', df_test)]:
    for clase in ['benignas', 'malignas']:
        carpeta_clase = os.path.join(carpeta_destino, conjunto, clase)
        os.makedirs(carpeta_clase, exist_ok=True)

    for _, fila in df_conjunto.iterrows():
        nombre_imagen = fila['image_id'] + ".jpg"
        etiqueta = fila['etiqueta']
        origen = os.path.join(carpeta_origen, nombre_imagen)
        destino = os.path.join(carpeta_destino, conjunto, etiqueta, nombre_imagen)

        if os.path.exists(origen):
            shutil.copy2(origen, destino)
        else:
            print(f'Imagen no encontrada: {origen}')


Colocar malignas en una sola carpeta

In [2]:
def preparar_dataset_malignas(metadata_path="../data/raw/HAM10000_metadata.csv",
                              carpeta_origen="../data/raw/imagenes",
                              carpeta_destino="../data/malignas_classes",
                              test_size=0.1,
                              random_state=42):
    # Leer metadata
    df = pd.read_csv(metadata_path)

    # Definir clases benignas y malignas
    clases_benignas = {'nv', 'bkl', 'df'}
    clases_malignas = set(df['dx'].unique()) - clases_benignas

    # Añadir columna binaria
    df['etiqueta'] = df['dx'].apply(lambda x: 'benignas' if x in clases_benignas else 'malignas')

    # Dividir en train y test estratificado por etiqueta
    df_train, df_test = train_test_split(df, test_size=test_size,
                                         stratify=df['etiqueta'], random_state=random_state)

    # Crear carpetas destino
    for conjunto, df_conjunto in [('train', df_train), ('test', df_test)]:
        # Carpetas para benignas (una sola)
        carpeta_benigna = os.path.join(carpeta_destino, conjunto, "benignas")
        os.makedirs(carpeta_benigna, exist_ok=True)

        # Carpetas para cada clase maligna
        for clase in clases_malignas:
            carpeta_clase = os.path.join(carpeta_destino, conjunto, clase)
            os.makedirs(carpeta_clase, exist_ok=True)

        # Copiar imágenes
        for _, fila in df_conjunto.iterrows():
            nombre_imagen = fila['image_id'] + ".jpg"
            origen = os.path.join(carpeta_origen, nombre_imagen)

            if fila['etiqueta'] == 'benignas':
                destino = os.path.join(carpeta_benigna, nombre_imagen)
            else:
                destino = os.path.join(carpeta_destino, conjunto, fila['dx'], nombre_imagen)

            if os.path.exists(origen):
                shutil.copy2(origen, destino)
            else:
                print(f"Imagen no encontrada: {origen}")
preparar_dataset_malignas()